<a href="https://colab.research.google.com/github/lkarjun/Data-Science-from-Scratch/blob/master/gradient_boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

df = pd.read_csv('concrete_data.csv')
df.head()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
from sklearn.model_selection import train_test_split

x_t, x_test, y_t, y_test = train_test_split(df.drop('csMPa', axis=1), df.csMPa)

In [6]:
from sklearn.tree import DecisionTreeRegressor
def tree(depth): return DecisionTreeRegressor(max_depth=depth)

In [8]:
tree_reg1 = tree(3)
tree_reg1.fit(x_t, y_t)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=3,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [9]:
y2 = y_t - tree_reg1.predict(x_t)
y2[:10]

607    -9.376071
906    -4.553611
463    15.056455
496     8.449592
704    -3.338941
939     4.923929
477   -20.623611
608   -19.053545
555     0.143929
730    -1.623545
Name: csMPa, dtype: float64

In [10]:
tree_reg2 = tree(4)
tree_reg2.fit(x_t, y2)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=4,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [12]:
y3 = y2 - tree_reg2.predict(x_t)
y3[:10]

607    -3.539143
906    -8.574365
463    11.035701
496     9.636987
704     3.151429
939     6.111324
477    -9.340674
608   -13.216617
555    -3.876825
730     4.213383
Name: csMPa, dtype: float64

In [14]:
tree_reg3 = tree(5)
tree_reg3.fit(x_t, y3)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=5,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [16]:
y_pred = sum(trees.predict(x_test) for trees in (tree_reg1, tree_reg2, tree_reg3))

In [22]:
from sklearn.metrics import r2_score

r2_score(y_test, y_pred)

0.7153959034811892

In [23]:
from sklearn.ensemble import GradientBoostingRegressor

In [51]:
gbr = GradientBoostingRegressor(max_depth=5, n_estimators=300, learning_rate=0.1)
gbr.fit(x_t, y_t)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=5,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=300,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [52]:
pred = gbr.predict(x_test)
r2_score(pred, y_test)

0.8979199975689336

In [56]:
pd.Series(gbr.feature_importances_, index = x_t.columns)

cement              0.259314
slag                0.072398
flyash              0.026765
water               0.058320
superplasticizer    0.127547
coarseaggregate     0.016541
fineaggregate       0.047619
age                 0.391496
dtype: float64

In [57]:
from sklearn.model_selection import GridSearchCV

In [60]:
parameters = {'n_estimators': [1,5, 10, 50, 100, 150, 200, 250, 300, 350, 400, 450, 499, 500, 504]}

In [61]:
gridSearch = GridSearchCV(estimator=gbr, param_grid=parameters, cv=3)

In [62]:
gridSearch.fit(x_t, y_t)

GridSearchCV(cv=3, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=5,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=300,
                                                 n_iter_no_change=None,
                 

In [63]:
gridSearch.best_params_

{'n_estimators': 300}

In [64]:
gbr_best = GradientBoostingRegressor(max_depth=5, n_estimators=gridSearch.best_params_['n_estimators'])

In [65]:
gbr_best.fit(x_t, y_t)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=5,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=300,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [66]:
pred = gbr.predict(x_test)
r2_score(pred, y_test)

0.8979199975689336